In [ ]:
import threading
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Simulated optimization process (runs in background)
progress = []
def optimization_process():
    for i in range(100):
        time.sleep(0.1)  # Simulate work
        progress.append(np.sin(i/10) + np.random.randn()*0.1)  # Simulate metric

# Start the background process
thread = threading.Thread(target=optimization_process)
thread.start()

# Monitor and plot progress
plt.ion()
fig, ax = plt.subplots()
line, = ax.plot([], [])
ax.set_ylim(-2, 2)
ax.set_xlim(0, 100)
ax.set_xlabel("Iteration")
ax.set_ylabel("Metric")

while thread.is_alive() or len(progress) < 100:
    line.set_data(range(len(progress)), progress)
    ax.set_xlim(0, max(100, len(progress)))
    clear_output(wait=True)
    display(fig)
    plt.pause(0.1)

# Final plot
line.set_data(range(len(progress)), progress)
clear_output(wait=True)
display(fig)
plt.ioff()

In [ ]:
import sys
import threading
import subprocess
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

progress_file = "progress.txt"

# Start the optimization process in a thread
proc = subprocess.Popen([sys.executable, "optimization_process.py", progress_file])

# File-watching thread: reads progress from the file
progress = []
def watch_progress_file():
    last_len = 0
    while True:
        ret = proc.poll()
        if ret is not None:
            break
        try:
            with open(progress_file) as f:
                lines = f.readlines()
                # Only update if new lines are added
                if len(lines) > last_len:
                    progress[:] = [float(line.strip()) for line in lines if line.strip()]
                    last_len = len(lines)
        except Exception:
            print("Error reading progress file.")
        if last_len >= 100:
            break
        time.sleep(0.1)

# Start the file-watching thread
watch_thread = threading.Thread(target=watch_progress_file)
watch_thread.start()

# Plotting loop (main thread)
plt.ion()
fig, ax = plt.subplots()
line, = ax.plot([], [])
ax.set_ylim(-2, 2)
ax.set_xlim(0, 100)
ax.set_xlabel("Iteration")
ax.set_ylabel("Metric")

while watch_thread.is_alive() or len(progress) < 100:
    y = list(progress)
    if y:
        line.set_data(range(len(y)), y)
        ax.set_xlim(0, max(100, len(y)))
    clear_output(wait=True)
    display(fig)
    plt.pause(0.1)

# Final plot
y = list(progress)
if y:
    line.set_data(range(len(y)), y)
    ax.set_xlim(0, max(100, len(y)))
clear_output(wait=True)
display(fig)
plt.ioff()